# Redis Testing n Stuff

In [9]:
import sys
import json
# import redis
from dataclasses import asdict
from datetime import datetime
import time

# from qdrant_client import QdrantClient
from pathlib import Path
from typing import Optional, Sequence
from tqdm.asyncio import tqdm

# Add parent directory to path to import from implementation package
# Notebooks are in implementation/notebooks/, so we go up two levels to project root
sys.path.insert(0, str(Path().resolve().parent.parent))

from implementation.classes.movie import BaseMovie
from implementation.classes.enums import EntityCategory, Genre
from implementation.classes.schemas import MetadataFilters, ExtractedEntitiesResponse, LexicalCandidate, ExtractedEntityData
from implementation.classes.languages import Language
from implementation.classes.watch_providers import FILTERABLE_WATCH_PROVIDERS_MAP
from db.ingest_movie import ingest_movie, _build_qdrant_payload, ingest_movie_to_qdrant, ingest_movies_to_qdrant_batched
from implementation.misc.helpers import tokenize_title_phrase, create_watch_provider_offering_key
from implementation.classes.watch_providers import FILTERABLE_WATCH_PROVIDER_IDS
from db.postgres import (
    pool, 
    refresh_title_token_doc_frequency,
    refresh_movie_popularity_scores, 
    fetch_movie_cards, 
    batch_upsert_genre_dictionary,
    batch_upsert_language_dictionary, 
    batch_upsert_maturity_dictionary, 
    batch_upsert_provider_dictionary,
    batch_upsert_watch_method_dictionary
)
from db.tmdb import fetch_trending_movie_ids
from db.redis import get_redis_client, init_redis
from db.trending_movies import compute_trending_score, refresh_trending_scores
from db.lexical_search import lexical_search, _resolve_all_title_tokens

# # Redis
# r = redis.Redis(host="localhost", port=6379, decode_responses=True)

# # Qdrant
# qdrant = QdrantClient(host="localhost", port=6333)

In [10]:
await init_redis()

In [2]:
trending_movies_ordered = await fetch_trending_movie_ids()

trending_movies_ordered

pages_needed: 25


[799882,
 1159559,
 1290821,
 1032892,
 858024,
 1236153,
 1272837,
 1327819,
 1317288,
 680493,
 1084242,
 1316092,
 1297842,
 1428862,
 1368166,
 1084244,
 1214931,
 1159831,
 1242898,
 804370,
 687163,
 1054867,
 1327688,
 1268609,
 1242419,
 1124566,
 1242937,
 83533,
 1233413,
 1261188,
 425274,
 650524,
 1171145,
 1263012,
 467905,
 840464,
 1208348,
 1315303,
 1327690,
 1226863,
 1234731,
 1088434,
 1246049,
 931285,
 1375831,
 1389149,
 701387,
 1220564,
 1159206,
 1462229,
 639988,
 911430,
 1061474,
 1317149,
 967941,
 1233506,
 1445363,
 1228246,
 1449400,
 1062722,
 1241983,
 1110034,
 1140498,
 1235746,
 1243006,
 1043197,
 1193128,
 1151272,
 1371185,
 1199193,
 1306368,
 1168190,
 1419406,
 1078605,
 1309725,
 1317672,
 533533,
 1100988,
 1456349,
 1358491,
 1266127,
 1198994,
 648878,
 1086260,
 1259102,
 878608,
 1128377,
 1244975,
 1497970,
 1206008,
 1575337,
 1264805,
 1116201,
 696393,
 1083381,
 1618784,
 1234821,
 1273221,
 1422562,
 617126,
 1311031,
 1119449,
 

In [6]:
n = len(trending_movies_ordered,)
for i, movie_id in enumerate(trending_movies_ordered[:10], start=1):
    score = compute_trending_score(i, n)
    print(f"movie_id: {movie_id}, score: {score}")






movie_id: 799882, score: 0.9552786404500042
movie_id: 1159559, score: 0.9367544467966324
movie_id: 1290821, score: 0.9225403330758517
movie_id: 1032892, score: 0.9105572809000084
movie_id: 858024, score: 0.9
movie_id: 1236153, score: 0.8904554884989668
movie_id: 1272837, score: 0.8816784043380077
movie_id: 1327819, score: 0.8735088935932649
movie_id: 1317288, score: 0.8658359213500126
movie_id: 680493, score: 0.8585786437626906


In [11]:
await refresh_trending_scores()

pages_needed: 25


500

In [15]:
ids_to_check = [1171145, 1119449, 1198994, 1445363, 1287141, 1316092, 1368166, 1297842]

for movie_id in ids_to_check:
    rank = trending_movies_ordered.index(movie_id) + 1
    print(f"movie_id: {movie_id}, rank: {rank}")

movie_id: 1171145, rank: 33
movie_id: 1119449, rank: 102
movie_id: 1198994, rank: 82
movie_id: 1445363, rank: 57
movie_id: 1287141, rank: 120
movie_id: 1316092, rank: 12
movie_id: 1368166, rank: 15
movie_id: 1297842, rank: 13
